# Lab 5: Hybrid Orchestration & Multi-Turn Conversation

**Purpose:** Advanced multi-turn conversation management using the three-tier hybrid routing system (Local → APIM → Foundry Agents) with intelligent context preservation and seamless model switching.

## Overview

This lab combines the hybrid routing system from Lab 4 with sophisticated conversation management:
- **Three-tier routing**: Local, APIM Model Router, Foundry Agents
- **Context preservation**: Chat history maintained across all routing targets
- **Intelligent conversation**: Context-aware routing decisions
- **Seamless switching**: Transparent model changes during conversation
- **Enterprise features**: Session management and conversation analytics

## Step 5.1: Environment Setup and Import Hybrid Router

In [ ]:
import os
import sys
import time
import json
from datetime import datetime
from typing import Dict, List, Tuple, Optional
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Add parent directory for module imports
sys.path.append(os.path.dirname(os.getcwd()))

# Load environment variables
load_dotenv()

print("✅ Environment setup complete")
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Add parent directory to path for module imports
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
modules_dir = os.path.join(parent_dir, 'modules')
if modules_dir not in sys.path:
    sys.path.append(modules_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

print(parent_dir)
print(modules_dir)

In [ ]:
# Import the hybrid router from Lab 4
try:
    from modules.hybrid_router import HybridFoundryAPIMRouter, HybridRouterConfig, create_hybrid_router_from_env
    router_available = True
    print("✅ Hybrid router module imported successfully")
except ImportError as e:
    router_available = False
    print(f"❌ Hybrid router not available: {e}")
    print("Please complete Lab 4 first or ensure modules are properly configured")

# Try to create the hybrid router from environment
if router_available:
    try:
        hybrid_router = create_hybrid_router_from_env()
        print("✅ Hybrid router initialized from environment")
        
        # Show available capabilities
        capabilities = hybrid_router.get_system_capabilities()
        print(f"\n📊 Available routing targets:")
        for target, available in capabilities['available_targets'].items():
            status = "✅" if available else "❌"
            print(f"   {target}: {status}")
            
    except Exception as e:
        print(f"⚠️ Failed to create hybrid router: {e}")
        print("Will create fallback router")
        hybrid_router = None
else:
    hybrid_router = None

## Step 5.2: Advanced Conversation Management System

In [ ]:
class HybridConversationManager:
    """Advanced conversation manager for three-tier hybrid routing system."""
    
    def __init__(self, hybrid_router=None, max_history=15, session_id=None):
        self.hybrid_router = hybrid_router
        self.max_history = max_history
        self.session_id = session_id or f"session_{int(time.time())}"
        
        # Conversation state
        self.chat_history = []  # Full conversation history
        self.conversation_stats = {
            'total_exchanges': 0,
            'local_responses': 0,
            'apim_responses': 0,
            'foundry_responses': 0,
            'azure_responses': 0,
            'mock_responses': 0,
            'model_switches': 0,
            'fallback_uses': 0,
            'session_start': datetime.now(),
            'last_source': None
        }
        
        # Validate router availability
        self.router_available = self._validate_router()
        
        print(f"🎭 Hybrid Conversation Manager initialized")
        print(f"   Session ID: {self.session_id}")
        print(f"   Max history: {self.max_history} exchanges")
        print(f"   Router status: {'✅ Available' if self.router_available else '⚠️ Mock mode'}")
    
    def _validate_router(self):
        """Validate if the hybrid router is properly configured."""
        if not self.hybrid_router:
            return False
        
        try:
            # Test basic router functionality
            capabilities = self.hybrid_router.get_system_capabilities()
            available_targets = capabilities.get('available_targets', {})
            
            # Check if at least one target is available
            has_working_target = any(available_targets.values())
            
            if not has_working_target:
                print("⚠️ No routing targets available, will use mock responses")
                return False
                
            return True
            
        except Exception as e:
            print(f"⚠️ Router validation failed: {e}")
            return False
    
    def _create_mock_response(self, user_message: str) -> str:
        """Create a mock response when routing fails."""
        import random
        
        # Simple keyword-based mock routing
        message_lower = user_message.lower()
        
        if any(word in message_lower for word in ['hello', 'hi', 'hey', 'greet']):
            responses = [
                "[MOCK-LOCAL] Hello! I'm a mock local assistant.",
                "[MOCK-LOCAL] Hi there! This is a simulated local response.",
                "[MOCK-LOCAL] Hey! Mock local model responding."
            ]
            source = "mock-local"
        elif any(word in message_lower for word in ['analyze', 'complex', 'detailed', 'comprehensive']):
            responses = [
                "[MOCK-CLOUD] This is a simulated complex analysis from the cloud model. In a real scenario, this would provide detailed insights.",
                "[MOCK-FOUNDRY] Mock Foundry Agent response: I would analyze this comprehensively using advanced AI capabilities.",
                "[MOCK-APIM] Simulated APIM routing to enterprise model for complex analysis."
            ]
            source = "mock-cloud"
        elif any(word in message_lower for word in ['calculate', 'math', '+', '-', '*', '/']):
            responses = [
                "[MOCK-LOCAL] Simple calculation handled by mock local model.",
                "[MOCK-LOCAL] Mock local response for mathematical query."
            ]
            source = "mock-local"
        else:
            responses = [
                "[MOCK-LOCAL] Mock response from local model for general query.",
                "[MOCK-CLOUD] Mock response from cloud model for this query.",
                "[MOCK-APIM] Simulated APIM routing response."
            ]
            source = "mock-general"
        
        response = random.choice(responses)
        return f"{response} [User asked: '{user_message[:50]}...']"
    
    def _format_history_for_model(self, include_system=True) -> List[Dict]:
        """Format chat history for model consumption."""
        messages = []
        
        if include_system:
            messages.append({
                "role": "system",
                "content": "You are an AI assistant in a hybrid routing system. Maintain conversation context and provide helpful responses."
            })
        
        # Add recent conversation history
        for exchange in self.chat_history[-self.max_history:]:
            messages.append({"role": "user", "content": exchange['user_message']})
            messages.append({"role": "assistant", "content": exchange['response']})
        
        return messages
    
    def _update_stats(self, source: str, was_fallback: bool = False):
        """Update conversation statistics."""
        self.conversation_stats['total_exchanges'] += 1
        
        # Track by source type
        source_lower = source.lower()
        if 'local' in source_lower:
            self.conversation_stats['local_responses'] += 1
        elif 'apim' in source_lower:
            self.conversation_stats['apim_responses'] += 1
        elif 'foundry' in source_lower:
            self.conversation_stats['foundry_responses'] += 1
        elif 'azure' in source_lower:
            self.conversation_stats['azure_responses'] += 1
        elif 'mock' in source_lower:
            self.conversation_stats['mock_responses'] += 1
        
        # Track model switches
        if self.conversation_stats['last_source'] and self.conversation_stats['last_source'] != source:
            self.conversation_stats['model_switches'] += 1
        
        # Track fallback usage
        if was_fallback:
            self.conversation_stats['fallback_uses'] += 1
        
        self.conversation_stats['last_source'] = source

    def chat(self, user_message: str, show_routing=False) -> str:
        """Process user message through hybrid routing with conversation context."""
        
        # Use mock response if router is not available
        if not self.router_available:
            if show_routing:
                print("🎯 Using mock routing (router unavailable)")
            
            mock_response = self._create_mock_response(user_message)
            
            # Extract source information from mock response
            if '[MOCK-' in mock_response:
                source_start = mock_response.find('[MOCK-') + 1
                source_end = mock_response.find(']', source_start)
                source = mock_response[source_start:source_end]
                clean_response = mock_response[source_end+1:].strip()
            else:
                source = "mock-unknown"
                clean_response = mock_response
            
            # Update statistics
            self._update_stats(source, was_fallback=True)
            
            # Store in history
            exchange = {
                'timestamp': datetime.now(),
                'user_message': user_message,
                'response': clean_response,
                'source': source,
                'was_fallback': True,
                'exchange_number': len(self.chat_history) + 1
            }
            
            self.chat_history.append(exchange)
            return mock_response
        
        # Add context-aware routing hints
        context_info = ""
        if len(self.chat_history) > 0:
            context_info = f" [Context: {len(self.chat_history)} previous exchanges]"
            user_message_with_context = f"{user_message}{context_info}"
        else:
            user_message_with_context = user_message
        
        # Route and get response with error handling
        try:
            response = self.hybrid_router.route(user_message_with_context, show_reasoning=show_routing)
            
            # Extract source information
            source = "unknown"
            clean_response = response
            was_fallback = False
            
            # Parse response format
            if response.startswith('[') and ']' in response:
                source_end = response.find(']')
                if source_end != -1:
                    source = response[1:source_end]
                    clean_response = response[source_end+1:].strip()
            elif response.startswith('ERROR') or 'error' in response.lower():
                # Handle error responses
                source = "ERROR"
                was_fallback = True
                clean_response = "I apologize, but I'm experiencing technical difficulties. Please try again or rephrase your question."
            
            # Detect if fallback was used
            if not was_fallback:
                was_fallback = '*' in source or 'fallback' in source.lower() or 'error' in source.lower()
            
            # Update statistics
            self._update_stats(source, was_fallback)
            
            # Store in history
            exchange = {
                'timestamp': datetime.now(),
                'user_message': user_message,
                'response': clean_response,
                'source': source,
                'was_fallback': was_fallback,
                'exchange_number': len(self.chat_history) + 1
            }
            
            self.chat_history.append(exchange)
            
            # Show routing info if requested
            if show_routing:
                print(f"🎯 Routed to: {source}")
                if was_fallback:
                    print(f"🔄 Fallback was used")
            
            return response
            
        except Exception as e:
            error_msg = f"❌ Chat error: {str(e)}"
            print(error_msg)
            
            # Create fallback response
            fallback_response = f"[ERROR] I encountered an issue processing your request. This might be due to connectivity or configuration problems. Your message was: '{user_message}'"
            
            # Update statistics for error
            self._update_stats("ERROR", was_fallback=True)
            
            # Store error in history
            exchange = {
                'timestamp': datetime.now(),
                'user_message': user_message,
                'response': "I apologize, but I'm experiencing technical difficulties.",
                'source': "ERROR",
                'was_fallback': True,
                'exchange_number': len(self.chat_history) + 1
            }
            
            self.chat_history.append(exchange)
            return fallback_response
    
    def get_conversation_summary(self) -> Dict:
        """Get comprehensive conversation analytics."""
        if not self.chat_history:
            return {"message": "No conversation history available"}
        
        # Calculate session duration
        session_duration = datetime.now() - self.conversation_stats['session_start']
        
        # Analyze routing patterns
        total = self.conversation_stats['total_exchanges']
        routing_distribution = {
            'local': (self.conversation_stats['local_responses'] / total * 100) if total > 0 else 0,
            'apim': (self.conversation_stats['apim_responses'] / total * 100) if total > 0 else 0,
            'foundry': (self.conversation_stats['foundry_responses'] / total * 100) if total > 0 else 0,
            'azure': (self.conversation_stats['azure_responses'] / total * 100) if total > 0 else 0
        }
        
        return {
            'session_info': {
                'session_id': self.session_id,
                'duration': str(session_duration).split('.')[0],
                'total_exchanges': total,
                'conversation_length': len(self.chat_history)
            },
            'routing_stats': {
                'distribution': routing_distribution,
                'model_switches': self.conversation_stats['model_switches'],
                'fallback_uses': self.conversation_stats['fallback_uses'],
                'current_source': self.conversation_stats['last_source']
            },
            'conversation_flow': [
                {
                    'exchange': ex['exchange_number'],
                    'source': ex['source'],
                    'fallback': ex['was_fallback'],
                    'timestamp': ex['timestamp'].strftime('%H:%M:%S')
                }
                for ex in self.chat_history[-5:]  # Last 5 exchanges
            ]
        }
    
    def export_conversation(self, filename: str = None) -> str:
        """Export conversation to JSON file."""
        if not filename:
            filename = f"conversation_{self.session_id}.json"
        
        export_data = {
            'session_info': {
                'session_id': self.session_id,
                'start_time': self.conversation_stats['session_start'].isoformat(),
                'export_time': datetime.now().isoformat(),
                'total_exchanges': len(self.chat_history)
            },
            'conversation': [
                {
                    'exchange_number': ex['exchange_number'],
                    'timestamp': ex['timestamp'].isoformat(),
                    'user_message': ex['user_message'],
                    'response': ex['response'],
                    'source': ex['source'],
                    'was_fallback': ex['was_fallback']
                }
                for ex in self.chat_history
            ],
            'statistics': self.conversation_stats
        }
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        return filename
    
    def clear_conversation(self):
        """Clear conversation history and reset statistics."""
        self.chat_history.clear()
        self.conversation_stats = {
            'total_exchanges': 0,
            'local_responses': 0,
            'apim_responses': 0,
            'foundry_responses': 0,
            'azure_responses': 0,
            'model_switches': 0,
            'fallback_uses': 0,
            'session_start': datetime.now(),
            'last_source': None
        }
        print(f"🧹 Conversation cleared for session {self.session_id}")

# Initialize the conversation manager
if hybrid_router:
    conversation_manager = HybridConversationManager(hybrid_router)
    print("✅ Hybrid Conversation Manager ready!")
else:
    conversation_manager = None
    print("❌ Cannot initialize conversation manager without hybrid router")

## Step 5.3: Interactive Multi-Turn Chat Session

In [ ]:
# Test conversation scenarios
test_conversation_scenarios = [
    # Scenario 1: Simple to Complex progression
    {
        "name": "Simple to Complex Progression",
        "messages": [
            "Hi there!",  # Should go to Local
            "What's the weather like?",  # Simple, Local
            "Explain machine learning algorithms in detail",  # Complex, Foundry/APIM
            "How would you implement this in an enterprise setting?"  # Enterprise, APIM
        ]
    },
    
    # Scenario 2: Multi-turn conversation with context
    {
        "name": "Context-Aware Conversation",
        "messages": [
            "I'm working on a Python project",  # Setup context
            "What's the best way to handle exceptions?",  # Technical question
            "Can you show me an example?",  # Follow-up with context
            "How would this scale in production?"  # Enterprise follow-up
        ]
    },
    
    # Scenario 3: Mixed complexity conversation
    {
        "name": "Mixed Complexity Chat",
        "messages": [
            "Hello",  # Simple greeting
            "Analyze the pros and cons of microservices architecture",  # Complex analysis
            "Thanks!",  # Simple response
            "What about security considerations?"  # Follow-up question
        ]
    }
]

def run_conversation_scenario(scenario: Dict, show_routing: bool = True):
    """Run a conversation scenario and display results."""
    if not conversation_manager:
        print("❌ Conversation manager not available")
        return
    
    print(f"\n{'='*60}")
    print(f"🎭 SCENARIO: {scenario['name']}")
    print(f"{'='*60}")
    
    # Clear previous conversation
    conversation_manager.clear_conversation()
    
    successful_exchanges = 0
    
    for i, message in enumerate(scenario['messages'], 1):
        print(f"\n🧑 User #{i}: {message}")
        print("-" * 50)
        
        try:
            response = conversation_manager.chat(message, show_routing=show_routing)
            print(f"🤖 Assistant: {response}")
            successful_exchanges += 1
        except Exception as e:
            print(f"❌ Error in exchange {i}: {e}")
            print(f"🤖 Assistant: I apologize, I'm having technical difficulties with that request.")
    
    # Show conversation summary
    print(f"\n📊 CONVERSATION SUMMARY:")
    try:
        summary = conversation_manager.get_conversation_summary()
        
        session_info = summary['session_info']
        routing_stats = summary['routing_stats']
        
        print(f"   Duration: {session_info['duration']}")
        print(f"   Successful exchanges: {successful_exchanges}/{len(scenario['messages'])}")
        print(f"   Total exchanges: {session_info['total_exchanges']}")
        print(f"   Model switches: {routing_stats['model_switches']}")
        print(f"   Fallbacks used: {routing_stats['fallback_uses']}")
        
        print(f"\n🎯 Routing Distribution:")
        for source, percentage in routing_stats['distribution'].items():
            if percentage > 0:
                print(f"   {source.capitalize()}: {percentage:.1f}%")
        
        print(f"\n🔄 Conversation Flow:")
        for flow in summary['conversation_flow']:
            status = "🔄" if flow['fallback'] else "✅"
            print(f"   {flow['exchange']}. [{flow['timestamp']}] → {flow['source']} {status}")
            
    except Exception as e:
        print(f"   ❌ Error generating summary: {e}")

# Run scenarios with enhanced error handling
if conversation_manager:
    print("🚀 Starting Enhanced Multi-Turn Conversation Testing...")
    
    # Test with simplified scenarios first
    simple_scenario = {
        "name": "Basic Connectivity Test",
        "messages": [
            "Hello",
            "How are you?",
            "Tell me about AI"
        ]
    }
    
    try:
        run_conversation_scenario(simple_scenario, show_routing=True)
        
        # If basic test works, run the full scenarios
        for scenario in test_conversation_scenarios:
            run_conversation_scenario(scenario, show_routing=True)
            time.sleep(1)  # Brief pause between scenarios
            
    except Exception as e:
        print(f"❌ Error running scenarios: {e}")
        print("💡 This might be due to router configuration issues.")
        print("   The system is now running in mock mode for demonstration.")
        
        # Run with mock responses
        print("\n🎭 Running demonstration with mock responses...")
        run_conversation_scenario(simple_scenario, show_routing=True)
        
else:
    print("❌ Cannot run conversation scenarios without conversation manager")

## Step 5.4: Interactive Chat Interface

In [ ]:
def interactive_chat_session():
    """Start an interactive chat session with hybrid routing."""
    if not conversation_manager:
        print("❌ Interactive chat not available without conversation manager")
        return
    
    print("🎭 HYBRID CHAT SESSION STARTED")
    print("=" * 50)
    print("💡 Features:")
    print("   • Three-tier routing: Local → APIM → Foundry Agents")
    print("   • Context preservation across model switches")
    print("   • Real-time routing decisions")
    print("   • Conversation analytics")
    print()
    print("🎯 Commands:")
    print("   'quit' or 'exit' - End session")
    print("   'stats' - Show conversation statistics")
    print("   'history' - Show recent conversation")
    print("   'clear' - Clear conversation history")
    print("   'export' - Export conversation to file")
    print("=" * 50)
    
    # Start fresh conversation
    conversation_manager.clear_conversation()
    
    while True:
        try:
            # Get user input
            user_input = input("\n🧑 You: ").strip()
            
            # Handle commands
            if user_input.lower() in ['quit', 'exit']:
                print("👋 Ending chat session...")
                break
            elif user_input.lower() == 'stats':
                summary = conversation_manager.get_conversation_summary()
                print("\n📊 CONVERSATION STATISTICS:")
                print(f"   Session: {summary['session_info']['session_id']}")
                print(f"   Duration: {summary['session_info']['duration']}")
                print(f"   Exchanges: {summary['session_info']['total_exchanges']}")
                print(f"   Model switches: {summary['routing_stats']['model_switches']}")
                print(f"   Current source: {summary['routing_stats']['current_source']}")
                continue
            elif user_input.lower() == 'history':
                print("\n📜 RECENT CONVERSATION:")
                for ex in conversation_manager.chat_history[-3:]:
                    print(f"   🧑 {ex['user_message']}")
                    print(f"   🤖 [{ex['source']}] {ex['response'][:100]}...")
                continue
            elif user_input.lower() == 'clear':
                conversation_manager.clear_conversation()
                print("🧹 Conversation history cleared")
                continue
            elif user_input.lower() == 'export':
                filename = conversation_manager.export_conversation()
                print(f"💾 Conversation exported to: {filename}")
                continue
            elif not user_input:
                continue
            
            # Process message through hybrid routing
            print("🤖 Assistant: ", end="", flush=True)
            response = conversation_manager.chat(user_input, show_routing=False)
            print(response)
            
            # Show routing info briefly
            if conversation_manager.chat_history:
                last_exchange = conversation_manager.chat_history[-1]
                source_info = f"[{last_exchange['source']}]"
                if last_exchange['was_fallback']:
                    source_info += " 🔄"
                print(f"   {source_info}")
            
        except KeyboardInterrupt:
            print("\n\n👋 Chat session interrupted by user")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
    
    # Final summary
    if conversation_manager.chat_history:
        print("\n📊 FINAL SESSION SUMMARY:")
        summary = conversation_manager.get_conversation_summary()
        session_info = summary['session_info']
        routing_stats = summary['routing_stats']
        
        print(f"   Duration: {session_info['duration']}")
        print(f"   Total exchanges: {session_info['total_exchanges']}")
        print(f"   Model switches: {routing_stats['model_switches']}")
        print(f"   Fallback uses: {routing_stats['fallback_uses']}")
        
        print(f"\n🎯 Routing Distribution:")
        for source, percentage in routing_stats['distribution'].items():
            if percentage > 0:
                print(f"   {source.capitalize()}: {percentage:.1f}%")

# Uncomment the line below to start interactive chat
# interactive_chat_session()

print("✅ Interactive chat interface ready!")
print("💡 Uncomment 'interactive_chat_session()' above to start chatting")

## Step 5.5: Advanced Features and Analytics

In [ ]:
# Advanced conversation analytics and optimization
def analyze_conversation_patterns():
    """Analyze conversation patterns and routing effectiveness."""
    if not conversation_manager or not conversation_manager.chat_history:
        print("❌ No conversation data available for analysis")
        return
    
    print("🔍 CONVERSATION PATTERN ANALYSIS")
    print("=" * 50)
    
    history = conversation_manager.chat_history
    total_exchanges = len(history)
    
    # Routing pattern analysis
    routing_sequences = []
    for i in range(len(history)):
        if i == 0:
            routing_sequences.append(f"START → {history[i]['source']}")
        else:
            routing_sequences.append(f"{history[i-1]['source']} → {history[i]['source']}")
    
    print(f"📊 Pattern Analysis:")
    print(f"   Total exchanges: {total_exchanges}")
    
    # Calculate routing efficiency
    successful_primary_routes = sum(1 for ex in history if not ex['was_fallback'])
    efficiency = (successful_primary_routes / total_exchanges * 100) if total_exchanges > 0 else 0
    
    print(f"   Primary routing success: {successful_primary_routes}/{total_exchanges} ({efficiency:.1f}%)")
    
    # Most common routing patterns
    from collections import Counter
    pattern_counts = Counter(routing_sequences)
    print(f"\n🔄 Most Common Routing Patterns:")
    for pattern, count in pattern_counts.most_common(3):
        print(f"   {pattern}: {count} times")
    
    # Response time analysis (if available)
    sources = [ex['source'] for ex in history]
    source_counts = Counter(sources)
    print(f"\n🎯 Source Distribution:")
    for source, count in source_counts.most_common():
        percentage = (count / total_exchanges * 100)
        print(f"   {source}: {count} ({percentage:.1f}%)")
    
    # Context effectiveness
    context_switches = conversation_manager.conversation_stats['model_switches']
    print(f"\n🔀 Context Management:")
    print(f"   Model switches: {context_switches}")
    print(f"   Switch rate: {(context_switches / max(total_exchanges-1, 1) * 100):.1f}%")
    
    return {
        'total_exchanges': total_exchanges,
        'routing_efficiency': efficiency,
        'pattern_distribution': dict(pattern_counts),
        'source_distribution': dict(source_counts),
        'context_switches': context_switches
    }

def generate_conversation_report():
    """Generate comprehensive conversation report."""
    if not conversation_manager:
        print("❌ No conversation manager available")
        return
    
    print("📋 COMPREHENSIVE CONVERSATION REPORT")
    print("=" * 60)
    
    # Basic session info
    summary = conversation_manager.get_conversation_summary()
    session_info = summary['session_info']
    routing_stats = summary['routing_stats']
    
    print(f"🆔 Session Information:")
    print(f"   Session ID: {session_info['session_id']}")
    print(f"   Duration: {session_info['duration']}")
    print(f"   Start Time: {conversation_manager.conversation_stats['session_start'].strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"   Total Exchanges: {session_info['total_exchanges']}")
    
    print(f"\n🎯 Routing Performance:")
    print(f"   Model Switches: {routing_stats['model_switches']}")
    print(f"   Fallback Uses: {routing_stats['fallback_uses']}")
    print(f"   Current Source: {routing_stats['current_source']}")
    
    print(f"\n📊 Distribution Analysis:")
    for source, percentage in routing_stats['distribution'].items():
        if percentage > 0:
            print(f"   {source.capitalize()}: {percentage:.1f}%")
    
    # Advanced analysis
    if conversation_manager.chat_history:
        analysis = analyze_conversation_patterns()
        
        print(f"\n⚡ Efficiency Metrics:")
        print(f"   Primary Routing Success: {analysis['routing_efficiency']:.1f}%")
        switch_rate = (analysis['context_switches'] / max(analysis['total_exchanges']-1, 1) * 100)
        print(f"   Context Switch Rate: {switch_rate:.1f}%")
        
    print(f"\n✅ Report generated successfully!")
    
    return summary

# Test the analytics functions
if conversation_manager and conversation_manager.chat_history:
    print("📊 Running conversation analytics...")
    analyze_conversation_patterns()
    print("\n" + "="*60)
    generate_conversation_report()
else:
    print("💡 Run some conversations first to see analytics in action!")
    print("   Use the test scenarios or interactive chat above")

## Step 5.6: Testing ConversationContextManager Alignment

**Purpose:** Test the enhanced ConversationContextManager to ensure it's aligned with HybridConversationManager and maintains conversation continuity.

In [ ]:
# Test the enhanced ConversationContextManager
import sys
import os

# Import the enhanced context manager
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from modules.context_manager import ConversationContextManager

print("🧪 Testing ConversationContextManager Alignment")
print("=" * 60)

# Create a test context manager
test_context_mgr = ConversationContextManager("test_session_001")

# Test conversation continuity with multiple exchanges
print("\n🔄 Testing Conversation Continuity:")
print("-" * 40)

# Exchange 1: Simple greeting
exchange1 = test_context_mgr.add_exchange(
    user_message="Hello! I'm new here.",
    ai_response="Welcome! I'm here to help you with any questions you have.",
    source="local",
    response_time=0.15,
    metadata={"context_used": False, "greeting_detected": True}
)

print(f"Exchange 1: ✅ Added (source: {exchange1['source']}, time: {exchange1['response_time']:.3f}s)")

# Exchange 2: Follow-up question  
exchange2 = test_context_mgr.add_exchange(
    user_message="What can you help me with?",
    ai_response="I can assist with various tasks including analysis, coding, explanations, and more. What specific area are you interested in?",
    source="local", 
    response_time=0.12,
    metadata={"context_used": True, "follow_up": True}
)

print(f"Exchange 2: ✅ Added (source: {exchange2['source']}, time: {exchange2['response_time']:.3f}s)")

# Exchange 3: Complex request (should route to different model)
exchange3 = test_context_mgr.add_exchange(
    user_message="Can you analyze the performance implications of different database indexing strategies for a large-scale e-commerce platform?",
    ai_response="This is a complex database architecture question. For large-scale e-commerce platforms, indexing strategy significantly impacts performance...",
    source="foundry",
    response_time=2.34,
    metadata={"context_used": True, "complex_analysis": True, "routing_reason": "Complex technical analysis"}
)

print(f"Exchange 3: ✅ Added (source: {exchange3['source']}, time: {exchange3['response_time']:.3f}s)")

# Exchange 4: Follow-up on previous complex topic
exchange4 = test_context_mgr.add_exchange(
    user_message="How would you implement that in a microservices architecture?", 
    ai_response="In a microservices architecture, database indexing becomes more complex due to data distribution. Here's how you'd approach it...",
    source="apim",
    response_time=1.89,
    metadata={"context_used": True, "microservices_context": True, "enterprise_query": True}
)

print(f"Exchange 4: ✅ Added (source: {exchange4['source']}, time: {exchange4['response_time']:.3f}s)")

# Exchange 5: Simple follow-up
exchange5 = test_context_mgr.add_exchange(
    user_message="Thanks, that was very helpful!",
    ai_response="You're welcome! I'm glad I could help explain the database indexing strategies for your use case.",
    source="local",
    response_time=0.08,
    metadata={"context_used": True, "gratitude": True}
)

print(f"Exchange 5: ✅ Added (source: {exchange5['source']}, time: {exchange5['response_time']:.3f}s)")

print(f"\n📊 Conversation Statistics:")
print(f"   Total exchanges: {len(test_context_mgr.chat_history)}")
print(f"   Model switches: {test_context_mgr.conversation_stats['model_switches']}")
print(f"   Current source: {test_context_mgr.conversation_stats['last_model_used']}")

# Test conversation context generation
recent_messages = [
    {"role": "user", "content": "Hello! I'm new here."},
    {"role": "assistant", "content": "Welcome! I'm here to help you with any questions you have."},
    {"role": "user", "content": "What can you help me with?"},
    {"role": "assistant", "content": "I can assist with various tasks including analysis, coding, explanations, and more."},
    {"role": "user", "content": "Can you analyze database indexing strategies?"}
]

context = test_context_mgr.get_conversation_context(recent_messages)
print(f"\n🧠 Generated Context Length: {len(context)} characters")
print(f"Context Preview: {context[:200]}...")

print("\n✅ ConversationContextManager test completed successfully!")

In [ ]:
# Test session summary and conversation flow alignment with lab5 HybridConversationManager
print("🔍 Testing Session Summary Alignment")
print("=" * 50)

# Get session summary in lab5 format
session_summary = test_context_mgr.get_session_summary()

print("📋 Session Information:")
session_info = session_summary['session_info']
for key, value in session_info.items():
    print(f"   {key}: {value}")

print("\n🎯 Routing Statistics:")
routing_stats = session_summary['routing_stats']
for key, value in routing_stats.items():
    if key == 'distribution':
        print(f"   {key}:")
        for source, percentage in value.items():
            if percentage > 0:
                print(f"     {source}: {percentage:.1f}%")
    else:
        print(f"   {key}: {value}")

print("\n🔄 Conversation Flow (Recent):")
for flow in session_summary['conversation_flow']:
    status = "🔄" if flow['fallback'] else "✅"
    print(f"   {flow['exchange']}. [{flow['timestamp']}] → {flow['source']} {status}")

# Test conversation continuity by simulating context usage
print(f"\n🧠 Testing Context Continuity:")
print("-" * 30)

# Get recent exchanges for context
recent_exchanges = test_context_mgr.get_recent_exchanges(3)
print(f"Recent exchanges count: {len(recent_exchanges)}")

for i, exchange in enumerate(recent_exchanges, 1):
    context_used = "🧠" if exchange['metadata'].get('context_used') else "❌"
    print(f"   {i}. [{exchange['timestamp'].strftime('%H:%M:%S')}] {exchange['source']} {context_used}")
    print(f"      User: {exchange['user_message'][:50]}...")
    print(f"      AI: {exchange['response'][:50]}...")

# Test that conversation history maintains proper order and continuity
print(f"\n🔗 Conversation Continuity Check:")
print("-" * 35)

for i in range(1, len(test_context_mgr.chat_history)):
    current = test_context_mgr.chat_history[i]
    previous = test_context_mgr.chat_history[i-1]
    
    # Check if context was properly used in follow-up questions
    if current['metadata'].get('context_used') and i > 1:
        print(f"   Exchange {current['exchange_number']}: Context used ✅")
        if current['source'] != previous['source']:
            print(f"     → Model switch: {previous['source']} → {current['source']}")
    
# Test export functionality for persistence
print(f"\n💾 Testing Export Functionality:")
export_file = f"test_conversation_{test_context_mgr.session_id}.json"
filename = test_context_mgr.export_conversation(export_file)
print(f"   Exported to: {filename}")

# Verify export content
import json
try:
    with open(filename, 'r') as f:
        exported_data = json.load(f)
    
    print(f"   ✅ Export successful!")
    print(f"   - Session ID: {exported_data['session_info']['session_id']}")
    print(f"   - Total exchanges: {exported_data['session_info']['total_exchanges']}")
    print(f"   - Conversation entries: {len(exported_data['conversation'])}")
    
    # Clean up test file
    os.remove(filename)
    print(f"   🧹 Cleanup: Test file removed")
    
except Exception as e:
    print(f"   ❌ Export verification failed: {e}")

print(f"\n✅ Session summary and continuity tests completed!")
print(f"🎯 ConversationContextManager is aligned with lab5 HybridConversationManager pattern!")

In [ ]:
# Test conversation continuity with backend API format
print("🔗 Testing Backend API Integration and Conversation Continuity")
print("=" * 65)

# Test that the ConversationContextManager works with the backend API format
test_backend_mgr = ConversationContextManager("backend_test_session")

# Simulate API exchanges like the backend would use them
print("🤖 Simulating Backend API Exchanges:")
print("-" * 40)

# Exchange 1: Initial greeting
user_msg1 = "Hello, I need help with my project"
ai_response1 = "Hi! I'd be happy to help you with your project. What type of project are you working on?"
metadata1 = {
    "strategy": "hybrid",
    "context_used": False,
    "routing_decision": "greeting_detected",
    "confidence": 0.95,
    "api_connected": True
}

exchange1 = test_backend_mgr.add_exchange(user_msg1, ai_response1, "local", 0.12, metadata1)
print(f"✅ Exchange 1: {exchange1['source']} - {exchange1['response_time']:.3f}s")

# Exchange 2: Follow-up with context
user_msg2 = "It's a web application for e-commerce"
ai_response2 = "Great! For e-commerce web applications, there are several key areas to consider. What specific aspect would you like help with?"
metadata2 = {
    "strategy": "hybrid", 
    "context_used": True,
    "routing_decision": "local_capability",
    "confidence": 0.88,
    "api_connected": True,
    "context_length": 1
}

exchange2 = test_backend_mgr.add_exchange(user_msg2, ai_response2, "local", 0.08, metadata2)
print(f"✅ Exchange 2: {exchange2['source']} - {exchange2['response_time']:.3f}s")

# Exchange 3: Complex technical question (should route to different model)
user_msg3 = "I need to design a scalable database architecture that can handle millions of transactions per day while ensuring ACID compliance and supporting both OLTP and OLAP workloads"
ai_response3 = "This is a complex database architecture challenge. For handling millions of daily transactions with ACID compliance and mixed OLTP/OLAP workloads, you'll need a sophisticated approach..."
metadata3 = {
    "strategy": "hybrid",
    "context_used": True, 
    "routing_decision": "complex_technical_analysis",
    "confidence": 0.92,
    "api_connected": True,
    "context_length": 2,
    "enterprise_query": True
}

exchange3 = test_backend_mgr.add_exchange(user_msg3, ai_response3, "foundry", 2.45, metadata3)
print(f"✅ Exchange 3: {exchange3['source']} - {exchange3['response_time']:.3f}s")

# Exchange 4: Follow-up referencing previous context
user_msg4 = "How would this integrate with the e-commerce platform I mentioned earlier?"
ai_response4 = "Excellent question! Integrating this database architecture with your e-commerce platform requires careful consideration of the transaction patterns..."
metadata4 = {
    "strategy": "hybrid",
    "context_used": True,
    "routing_decision": "context_aware_follow_up", 
    "confidence": 0.89,
    "api_connected": True,
    "context_length": 3,
    "references_previous_context": True
}

exchange4 = test_backend_mgr.add_exchange(user_msg4, ai_response4, "apim", 1.67, metadata4)
print(f"✅ Exchange 4: {exchange4['source']} - {exchange4['response_time']:.3f}s")

# Test conversation context generation
print(f"\n🧠 Testing Context Generation for API:")
print("-" * 35)

# Generate context like the backend API would
recent_messages = []
for ex in test_backend_mgr.chat_history:
    recent_messages.append({"role": "user", "content": ex['user_message']})
    recent_messages.append({"role": "assistant", "content": ex['response']})

context = test_backend_mgr.get_conversation_context(recent_messages[-6:], max_context_length=500)  # Last 3 exchanges
print(f"Context length: {len(context)} characters")
print(f"Context content:\n{context[:300]}...")

# Test session summary in backend-compatible format
print(f"\n📊 Session Summary (Backend Compatible):")
print("-" * 40)

summary = test_backend_mgr.get_session_summary()
print(f"Session ID: {summary['session_info']['session_id']}")
print(f"Total exchanges: {summary['session_info']['total_exchanges']}")
print(f"Model switches: {summary['routing_stats']['model_switches']}")

# Verify conversation continuity 
print(f"\n🔗 Conversation Continuity Verification:")
print("-" * 38)

for i, ex in enumerate(test_backend_mgr.chat_history):
    context_indicator = "🧠" if ex['metadata'].get('context_used') else "❌"
    switch_indicator = "🔄" if i > 0 and test_backend_mgr.chat_history[i-1]['source'] != ex['source'] else ""
    
    print(f"   {ex['exchange_number']}. [{ex['source']}] {context_indicator} {switch_indicator}")
    print(f"      Query: {ex['user_message'][:60]}...")
    
    # Check if the exchange properly references previous context
    if ex['metadata'].get('references_previous_context'):
        print(f"      🔗 References previous context")
    if ex['metadata'].get('context_length', 0) > 0:
        print(f"      📊 Context length: {ex['metadata']['context_length']} exchanges")

print(f"\n✅ Backend integration test completed!")
print(f"🎯 ConversationContextManager maintains proper conversation continuity!")
print(f"🔄 Model switches are tracked correctly: {test_backend_mgr.conversation_stats['model_switches']}")

# Test that conversation history is preserved properly
print(f"\n📝 Final Continuity Check:")
print(f"   - Conversation flows from greeting → follow-up → complex analysis → context-aware follow-up")
print(f"   - Context is properly maintained across model switches")  
print(f"   - Metadata preserves API-specific information")
print(f"   - Session management works for multi-turn conversations")

## 🎉 Lab 5 Complete!

### What You've Accomplished:
- ✅ **Advanced Hybrid Orchestration**: Integrated three-tier routing (Local → APIM → Foundry Agents) with conversation management
- ✅ **Context Preservation**: Maintained chat history across all routing targets and model switches
- ✅ **Multi-Turn Conversations**: Seamless context-aware conversations with intelligent routing
- ✅ **Real-Time Analytics**: Comprehensive conversation tracking and performance analysis
- ✅ **Interactive Interface**: Full-featured chat system with commands and session management

### Key Features Implemented:

**🎭 Hybrid Conversation Manager:**
- Three-tier routing integration with conversation context
- Automatic context preservation across model switches
- Real-time routing statistics and performance tracking
- Session management with export capabilities

**🤖 Intelligent Routing:**
- Context-aware routing decisions based on conversation history
- Fallback chain management with transparency
- Source tracking and routing pattern analysis
- Enterprise-grade APIM integration for business queries

**📊 Advanced Analytics:**
- Conversation pattern analysis and routing effectiveness
- Real-time performance metrics and efficiency tracking
- Export capabilities for conversation data
- Comprehensive reporting with detailed insights

**💬 Enterprise Chat Features:**
- Interactive chat interface with command system
- Session persistence and conversation export
- Multi-scenario testing with automated analysis
- Production-ready conversation management

### Architecture Benefits:

🏗️ **Scalable Design**: Modular architecture supports enterprise deployment  
🔄 **Seamless Switching**: Transparent model changes maintain conversation flow  
📈 **Analytics-Driven**: Real-time metrics enable continuous optimization  
🛡️ **Production-Ready**: Robust error handling and fallback mechanisms  
💰 **Cost-Effective**: Intelligent routing optimizes resource usage  

### Next Steps:
- **Lab 6**: Observability and telemetry integration
- **Lab 7**: Frontend chat interface development
- **Production**: Deploy with full monitoring and analytics

**Your enterprise-grade hybrid orchestration system with advanced conversation management is complete!** 🚀